In [ ]:
import psycopg2
import pandas as pd

# minute 데이터베이스 연결 (소스)
source_conn = psycopg2.connect(
    host='192.168.40.193',
    dbname='minute',
    user='postgres',
    password='mysecretpassword'
)

# mydb 데이터베이스 연결 (타겟)
target_conn = psycopg2.connect(
    host='192.168.40.193',
    dbname='mydb',
    user='postgres',
    password='mysecretpassword'
)

source_cursor = source_conn.cursor()
target_cursor = target_conn.cursor()

symbols = ["AAPL", "TSLA", "NVDA", "META", "GOOGL", "GOOG", "MSFT", "AMZN"]

for symbol in symbols:
    table_name = f"ticker_{symbol.lower()}"
    print(f"처리 중: {symbol} → {table_name}")
    
    # 1. mydb에 테이블 생성 (존재하지 않는 경우)
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        timestamp TIMESTAMP,
        open DECIMAL(10,4),
        high DECIMAL(10,4),
        low DECIMAL(10,4),
        close DECIMAL(10,4),
        volume BIGINT,
        PRIMARY KEY (timestamp)
    );
    """
    target_cursor.execute(create_table_query)
    
    # 2. minute 데이터베이스에서 데이터 조회
    select_query = """
    SELECT timestamp, open, high, low, close, volume
    FROM us_market_data
    WHERE symbol = %s
    """
    source_cursor.execute(select_query, (symbol,))
    data = source_cursor.fetchall()
    
    # 3. mydb에 데이터 삽입
    if data:
        insert_query = f"""
        INSERT INTO {table_name} (timestamp, open, high, low, close, volume)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON CONFLICT (timestamp) DO NOTHING;
        """
        target_cursor.executemany(insert_query, data)
        print(f"  → {len(data)}개 행 처리됨")
    else:
        print(f"  → 데이터 없음")

# 커밋 및 연결 종료
target_conn.commit()
source_conn.close()
target_conn.close()
print("모든 작업 완료!")

처리 중: AAPL → ticker_aapl
